In [55]:
import pandas as pd

In [56]:
df = pd.read_csv("LifeTrack_white_stork_clean.zip")

In [57]:
df.groupby("tag-local-identifier").count()

,event-id,timestamp,location-long,location-lat,ground-speed,height-above-ellipsoid,individual-taxon-canonical-name,eobs:temperature,individual-local-identifier,date
tag-local-identifier,,,,,,,,,,
3048,74819,74819,74819,74819,71264,71264,74819,71264,74819,74819
3050,61507,61507,61507,61507,59222,59222,61507,59222,61507,61507
3056,10445,10445,10445,10445,9310,9310,10445,9310,10445,10445
3075,7665,7665,7665,7665,6775,6775,7665,6775,7665,7665
3950,12325,12325,12325,12325,12310,12310,12325,12310,12325,12325
3951,98508,98508,98508,98508,98023,98023,98508,98023,98508,98508
3952,74290,74290,74290,74290,74285,74285,74290,74285,74290,74290
3953,22143,22143,22143,22143,21927,21927,22143,21927,22143,22143
3954,96382,96382,96382,96382,96112,96112,96382,96112,96382,96382


In [58]:
def set_migration_id(x):
    # pl -> afr
    s1_start = pd.Timestamp("2013-07-01")
    s1_end = pd.Timestamp("2013-12-01")
    
    # afr -> pl
    s2_start = pd.Timestamp("2014-02-01")
    s2_end = pd.Timestamp("2014-06-30")
    
    # pl -> afr
    s3_start = pd.Timestamp("2014-07-01")
    s3_end = pd.Timestamp("2014-12-01")
    
    # afr -> pl
    s4_start = pd.Timestamp("2015-02-01")
    s4_end = pd.Timestamp("2015-06-30")
    
    # pl -> afr
    s5_start = pd.Timestamp("2015-07-01")
    s5_end = pd.Timestamp("2015-12-01")
    
    date = pd.Timestamp(x["timestamp"])
    
    if s1_start <= date <= s1_end:
        return "pl_afr_1"
    elif s2_start <= date <= s2_end:
        return "afr_pl_2"
    elif s3_start <= date <= s3_end:
        return "pl_afr_3"
    elif s4_start <= date <= s4_end:
        return "afr_pl_4"
    elif s5_start <= date <= s5_end:
        return "pl_afr_5"
    else:
        return "kupa"

In [60]:
def get_processed_df_by_id(idx: int, df_all: pd.DataFrame):
    if idx != -1:  # -1 means all
        df_all = df_all.loc[df_all["tag-local-identifier"] == idx]

    # add migration id
    df_all["migration_id"] = df_all.apply(lambda row: set_migration_id(row), axis=1)
    df_all = df_all.loc[~df_all["migration_id"].str.contains("kupa")]
    
    df_all["stork_migration_id"] = df_all["tag-local-identifier"].astype(str) + "_" + df_all["migration_id"].astype(str)
    
    # remove points that are 100m or less close to prev point
    
    # select data for every 30mins
    # https://stackoverflow.com/questions/66908177/how-can-i-get-the-values-at-every-nth-hour-from-a-pandas-dataframe
    
    # get few columns only
    df_all = df_all[["stork_migration_id", "migration_id", "timestamp", "date", "location-long", "location-lat"]]
    
    return df_all

In [61]:
df_with_migration_id = get_processed_df_by_id(idx=-1, df_all=df)

C:\Users\lsaw9\AppData\Local\Temp\ipykernel_16004\1593745207.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["stork_migration_id"] = df_all["tag-local-identifier"].astype(str) + "_" + df_all["migration_id"].astype(str)


In [63]:
df_pl_afr_1 = df_with_migration_id.loc[df_with_migration_id["migration_id"].str.contains("pl_afr_1")]
df_afr_pl_2 = df_with_migration_id.loc[df_with_migration_id["migration_id"].str.contains("afr_pl_2")]
df_pl_afr_3 = df_with_migration_id.loc[df_with_migration_id["migration_id"].str.contains("pl_afr_3")]
df_afr_pl_4 = df_with_migration_id.loc[df_with_migration_id["migration_id"].str.contains("afr_pl_4")]
df_pl_afr_5 = df_with_migration_id.loc[df_with_migration_id["migration_id"].str.contains("pl_afr_5")]

In [64]:
df_pl_afr_1.to_csv("pl_afr_1.csv")
df_afr_pl_2.to_csv("afr_pl_2.csv")
df_pl_afr_3.to_csv("pl_afr_3.csv")
df_afr_pl_4.to_csv("afr_pl_4.csv")
df_pl_afr_5.to_csv("pl_afr_5.csv")

In [54]:
df_3075 = get_processed_df_by_id(3075, df)
df_3955 = get_processed_df_by_id(3955, df)
df_4557 = get_processed_df_by_id(4557, df)
df_4558 = get_processed_df_by_id(4558, df)
df_4559 = get_processed_df_by_id(4559, df)

df_3075.to_csv("stork_3075.csv")
df_3955.to_csv("stork_3955.csv")
df_4557.to_csv("stork_4557.csv")
df_4558.to_csv("stork_4558.csv")
df_4559.to_csv("stork_4559.csv")

C:\Users\lsaw9\AppData\Local\Temp\ipykernel_16004\743909388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["migration_id"] = df_all.apply(lambda row: set_migration_id(row), axis=1)
C:\Users\lsaw9\AppData\Local\Temp\ipykernel_16004\743909388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["migration_id"] = df_all.apply(lambda row: set_migration_id(row), axis=1)
C:\Users\lsaw9\AppData\Local\Temp\ipykernel_16004\743909388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a co